In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.metrics import classification_report

# Load the data
data = pd.read_csv('Yelp_Restaurant_Reviews.csv')

# Preprocessing the data using tfidf
tfidf = TfidfVectorizer(max_features=1000)
X = tfidf.fit_transform(data['Review Text']).toarray()

# Converting ratings to categorical sentiment
conditions = [
    data['Rating'] <= 2,  # Negative
    data['Rating'] == 3,  # Neutral
    data['Rating'] >= 4   # Positive
]
choices = [0, 1, 2]
y = np.select(conditions, choices)
y = to_categorical(y, num_classes=3)

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

# learning rate decay function to adjust the learning rate over the course of training
def lr_time_based_decay(epoch, lr):
    decay = 0.1
    return lr * 1 / (1 + decay * epoch)

# Model architecture : neural network model
model = Sequential([
    Dense(128, activation='relu', input_dim=1000),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'), 
    Dropout(0.5),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_time_based_decay)

# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, 
                    validation_data=(X_val, y_val), 
                    callbacks=[early_stopping, lr_scheduler])

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


# Printing classification report for the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
print(classification_report(y_test_labels, y_pred_labels, target_names=['Negative', 'Neutral', 'Positive']))


2024-05-03 17:42:27.786778: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 17:42:27.830875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 17:42:27.831877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 17:42:28.807964: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/10


2024-05-03 17:42:31.034285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-03 17:42:31.036476: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


187/187 [==============================] - 2s 4ms/step - loss: 0.9496 - accuracy: 0.6031 - val_loss: 0.6519 - val_accuracy: 0.7638 - lr: 0.0010
Epoch 2/10
187/187 [==============================] - 1s 3ms/step - loss: 0.5191 - accuracy: 0.8036 - val_loss: 0.5729 - val_accuracy: 0.7698 - lr: 9.0909e-04
Epoch 3/10
187/187 [==============================] - 1s 3ms/step - loss: 0.4338 - accuracy: 0.8355 - val_loss: 0.4295 - val_accuracy: 0.8319 - lr: 7.5758e-04
Epoch 4/10
187/187 [==============================] - 1s 3ms/step - loss: 0.3872 - accuracy: 0.8536 - val_loss: 0.3825 - val_accuracy: 0.8495 - lr: 5.8275e-04
Epoch 5/10
187/187 [==============================] - 1s 3ms/step - loss: 0.3586 - accuracy: 0.8594 - val_loss: 0.3790 - val_accuracy: 0.8542 - lr: 4.1625e-04
Epoch 6/10
187/187 [==============================] - 1s 3ms/step - loss: 0.3371 - accuracy: 0.8691 - val_loss: 0.3772 - val_accuracy: 0.8565 - lr: 2.7750e-04
Epoch 7/10
187/187 [==============================] - 1s 3ms/